# Basic tools use

Various LLM use cases can utilize function/tool calls to extend LLM capabilities. Draive comes with a dedicated solution to prepare tools and control its execution.

## Tool definition

Let's start by defining a simple tool. Tool is usually a function which is explained to LLM and can be requested to be used. Tools can require generation of arguments and have to return some value. Tools defined within draive are python async function annotated with `tool` wrapper.

In [1]:
from draive import tool


@tool # simply annotate function as a tool, tools can have arguments using basic types
async def current_time(location: str) -> str:
    # return result of tool execution, we are using fake time here
    return f"Time in {location} is 9:53:22"

## Tool use

After defining a tool we can use it in multiple scenarios to extend LLM capabilities. All of the tool arguments will be validated when calling. It includes type check and any additional validation if defined. You can still use it as a regular function, although, all tools have to be executed within draive context scope. 

In [2]:
from draive import ctx

async with ctx.scope("basics"):
    # we can still use it as a regular function
    # but it has to be executed within context scope
    print(await current_time(location="London"))

# await current_time(location="London") # error! out of context

Time in London is 9:53:22


The biggest benefit of defining a tool is when using LLM. We can tell the model to use available tools to extend its capabilities. Higher level interfaces automatically handle tool calls and going back with its result to LLM to receive the final result. We can see how it works within a simple text generator. Tools are provided within a `Toolbox` object which allows customizing tools execution. You can prepare a `Toolbox` each time it is used or reuse preconstructed one. We will use OpenAI GPT model as it natively supports tool use. Make sure to provide .env file with `OPENAI_API_KEY` key before running. 

In [3]:
from draive import Toolbox, generate_text, load_env
from draive.openai import OpenAIChatConfig, openai_lmm

load_env()

async with ctx.scope(
    "basics",
    # define used LMM to be OpenAI within the context
    openai_lmm(),
    OpenAIChatConfig(model="gpt-4o-mini"),
):
    result: str = await generate_text(
        instruction="You are a helpful assistant",
        input="What is the time in New York?",
        tools=[current_time],
    )

    print(result)

The current time in New York is 9:53 AM.


## Tool details

Tools can be customized and extended in various ways depending on use case. First of all we can customize tool arguments and help LLM to better understand how to use given tool.

In [4]:
from draive import Argument


@tool( # this time we will use additional arguments within tool annotation
    # we can define an alias used as the tool name when explaining it to LLM,
    # default name is the name of python function
    name="fun_fact",
    # additionally we can explain the tool purpose by using description
    description="Find a fun fact in a given topic",
)
async def customized(
    # we can also annotate arguments to provide even more details
    # and specify argument handling logic
    arg: str = Argument(
        # we can alias each argument name
        aliased="topic",
        # further describe its use
        description="Topic of a fact to find",
        # provide default value or default value factory
        default="random",
        # end more, including custom validators
    ),
) -> str:
    return f"{arg} is very funny on its own!"

# we can examine tool specification which is similar to
# how `State` and `DataModel` specification/schema is built
print(customized.specification)

{'name': 'fun_fact', 'description': 'Find a fun fact in a given topic', 'parameters': {'type': 'object', 'properties': {'topic': {'type': 'string', 'description': 'Topic of a fact to find'}}, 'required': []}}


## Toolbox

We have already used a Toolbox to ask for an extended LLM completion. However Toolbox allows us to specify some additional details regarding the tools execution like the tool calls limit or a tool suggestion.

In [5]:
async with ctx.scope(
    "basics",
    # define used LMM to be OpenAI within the context
    openai_lmm(),
    OpenAIChatConfig(model="gpt-4o-mini"),
):
    result: str = await generate_text(
        instruction="You are a funny assistant",
        input="What is the funny thing about LLMs?",
        tools=Toolbox.of(
            # we can define any number of tools within a toolbox
            current_time,
            # we can also force any given tool use within the first LLM call
            suggest=customized,
            # we can limit how many tool calls are allowed
            # before the final result is returned
            repeated_calls_limit=2,
        ),
    )

    print(result)

The funny thing about LLMs (Large Language Models) is that they can generate text that sounds remarkably coherent and intelligent, but they often have no idea what they’re talking about! It's like asking a parrot for life advice — it might sound profound, but it’s just mimicking what it has heard. Just imagine chatting with a robot that can quote Shakespeare while simultaneously forgetting your name!


## Metrics

All of the tool usage is automatically traced within scope metrics. We can see the details about their execution when using a logger:

In [6]:
from draive import setup_logging, usage_metrics_logger

setup_logging("basics")

async with ctx.scope(
    "basics",
    # define used LMM to be OpenAI within the context
    openai_lmm(),
    OpenAIChatConfig(model="gpt-4o-mini"),
    completion=usage_metrics_logger(),
):
    result: str = await generate_text(
        instruction="You are a funny assistant",
        input="What is the funny thing about LLMs?",
        # we will now be able to see what tools were used
        # and check the details about its execution
        tools=Toolbox.of(
            current_time,
            suggest=customized,
        ),
    )

    print(f"\nResult:\n{result}\n")

19/Dec/2024:17:51:57 +0000 [INFO] [basics] [185e42d863da45869acc9702d6220949] [basics] [2e773dc58ff14e37a6398552b5766a96] Started...
19/Dec/2024:17:51:57 +0000 [INFO] [basics] [b98eb4e0d921412c9e2bccce0330d510] [generate_text] [9497668f447647a19326bf8cb99251cb] Started...
19/Dec/2024:17:51:57 +0000 [INFO] [basics] [aeeffc3f63d04fb0940616b49f3a6548] [openai_lmm_invocation] [be45359b3a1b4e10b19fbed5825378ce] Started...
19/Dec/2024:17:51:57 +0000 [INFO] [basics] [aeeffc3f63d04fb0940616b49f3a6548] [openai_lmm_invocation] [be45359b3a1b4e10b19fbed5825378ce] ...finished after 0.45s
19/Dec/2024:17:51:57 +0000 [DEBUG] [basics] [b98eb4e0d921412c9e2bccce0330d510] [generate_text] [9497668f447647a19326bf8cb99251cb] Received text generation tool calls
19/Dec/2024:17:51:57 +0000 [INFO] [basics] [cb13d113f1824e1bb0cf8abeb2506c91] [fun_fact] [b2f682d55a01461e97409308ebab9e67] Started...
19/Dec/2024:17:51:57 +0000 [INFO] [basics] [cb13d113f1824e1bb0cf8abeb2506c91] [fun_fact] [b2f682d55a01461e97409308eba

19/Dec/2024:17:51:59 +0000 [INFO] [basics] [185e42d863da45869acc9702d6220949] [basics] [2e773dc58ff14e37a6398552b5766a96] Usage metrics:
@basics[2e773dc58ff14e37a6398552b5766a96](2.46s):
• TokenUsage:
|  + usage: 
|  |  + gpt-4o-mini: 
|  |  |  + input_tokens: 151
|  |  |  + output_tokens: 111
@generate_text[9497668f447647a19326bf8cb99251cb](2.46s):
|  • TokenUsage:
|  |  + usage: 
|  |  |  + gpt-4o-mini: 
|  |  |  |  + input_tokens: 151
|  |  |  |  + output_tokens: 111
|  @openai_lmm_invocation[be45359b3a1b4e10b19fbed5825378ce](0.45s):
|  |  • ArgumentsTrace:
|  |  |  + kwargs: 
|  |  |  |  + instruction: You are a funny assistant
|  |  |  |  + context: 
|  |  |  |  |  [0] content: 
|  |  |  |  |    parts: 
|  |  |  |  |      - text: What is the funny thing about LLMs?
|  |  |  |  |        meta: None
|  |  |  |  |  [1] requests: 
|  |  |  |  |    - identifier: call_H1TxwXL2YZvuOduib1Oh8H6r
|  |  |  |  |      tool: fun_fact
|  |  |  |  |      arguments: 
|  |  |  |  |        topic: lan